# <center>Naive Bayes</center>
## <div align='right'>Made by:</div>
**<div align='right'>Ihor Markevych</div>**

Implement a Naive Bayes classifier to apply it to the task of classifying handwritten digits. Files `mnist_train` and `mnist_test` contain training and test digits, together with their ground truth labels (first column). Each row in these files corresponds to a different digit.  
  
Each image is 28x28, hence there are 784 pixel in every image. Columns 2-785 in the data files correspond to the pixel intensity, a value between 0 to 255. Column 1 corresponds to the correct label for each digit.  
  
The pixel intensities will be converted to a single binary indicator feature ($F_i$) for each pixel. Specifically, if the intensity is smaller than 255/2 it will be mapped to a zero, otherwise to a one.

In [1]:
import pandas as pd
import numpy as np
import sklearn.metrics
from IPython.display import display, Markdown

In [2]:
train = pd.read_csv('./mnist_train.csv', header=None)
test = pd.read_csv('./mnist_test.csv', header=None)

In [3]:
trainY = train.iloc[:, 0]
trainX = train.iloc[:, 1:]

testY = test.iloc[:, 0]
testX = test.iloc[:, 1:]

In [4]:
trainX = trainX.applymap(lambda x: int(x > 255 / 2))
testX = testX.applymap(lambda x: int(x > 255 / 2))

In [5]:
class BinNaiveBayes:
    
    def __init__(self, smoothingCoef = 1):
        self.smoothingCoef = smoothingCoef
    
    def fit(self, trainX, trainY):
        self.prior = trainY.value_counts(sort=False)
        self.likelihood = np.array([np.sum(trainX.loc[trainY == i, :], axis=0) 
                                    for i in np.unique(trainY)])
        self.likelihood = (self.likelihood + self.smoothingCoef ) \
                            / (np.array(self.prior)[:, None] + 2 * self.smoothingCoef)
        
        self.prior /= len(trainY)
        
    def logProb(self, testXRow):
        return np.sum(
                np.log(
                    np.abs(
                        np.abs(np.array(testXRow) - 1) - self.likelihood
                    )
                ), axis=1) + np.log(self.prior)
        
    def predict(self, testX):
        if isinstance(testX, pd.Series):
            testX = pd.DataFrame(testX).T
        
        return [np.argmax(self.logProb(testXRow)) 
                for _, testXRow in testX.iterrows()]

In [6]:
NB = BinNaiveBayes()
NB.fit(trainX, trainY)

### Estimate the priors $P(class)$ based on the frequencies of different classes in the training set

In [7]:
round(NB.prior, 3)

0    0.099
1    0.112
2    0.099
3    0.102
4    0.097
5    0.090
6    0.099
7    0.104
8    0.098
9    0.099
Name: 0, dtype: float64

### Estimate the likelihoods $P(F_i|class)$ for every pixel location $i$ and for every digit class from $0$ to $9$.  
  
  
The likelihood estimate is  
$P(F_i = f|class)$ = (Number of times pixel $i$ has value $f$ in training examples from this class) / (Total number of training examples from this class)


The likelihoods should be smoothed to ensure that there are no zero counts. Laplace smoothing is a very simple method that increases the observation count of every value $f$ by some constant $k$. This corresponds to adding $k$ to the numerator above, and $k*V$ to the denominator (where $V$ is the number of possible values the feature can take on). The higher the value of $k$, the stronger the smoothing. Experiment with different integer values of $k$ from $1$ to $5$. 

In [8]:
for k in range (1, 6):
    NB = BinNaiveBayes(k)
    NB.fit(trainX, trainY)
    print(f'For k={str(k)}:')
    display(Markdown('$P(F_{682} = 0|class = 5)=$ ' + str(round(1 - NB.likelihood[5, 681], 3))))
    display(Markdown('$P(F_{772} = 1|class = 9)=$ ' + str(round(NB.likelihood[9, 771], 3))))
    print()

For k=1:


$P(F_{682} = 0|class = 5)=$ 0.85

$P(F_{772} = 1|class = 9)=$ 0.001


For k=2:


$P(F_{682} = 0|class = 5)=$ 0.85

...

$P(F_{682} = 0|class = 5)=$ 0.85

$P(F_{772} = 1|class = 9)=$ 0.002


For k=5:


$P(F_{682} = 0|class = 5)=$ 0.85

$P(F_{772} = 1|class = 9)=$ 0.002

### Maximum a posteriori (MAP) classification of test digits according to the learned Naive Bayes modeles.  
  
Suppose a test image has feature values $f_1, f_2, ... , f_{784}$. According to this model, the posterior probability (up to scale) of each class given the digit is given by:  
  
$P(class)P(f_1|class)(f_2|class)...P(f_{784}|class)$
  
Note that in order to avoid underflow, we need to work with the log of the above quantity:
  
$log P(class) + log P(f_1|class) + log P(f_2|class) + ... + log P(f_{784}|class)$

In [9]:
NB = BinNaiveBayes()
NB.fit(trainX, trainY)
print('For first test image, k = 1:')
display(Markdown('$log P(class = 5|f_1, f_2, ..., f_784)=$ ' + str(NB.logProb(testX.iloc[0])[5])))
display(Markdown('$log P(class = 7|f_1, f_2, ..., f_784)=$ ' + str(NB.logProb(testX.iloc[0])[7])))

For first test image, k = 1:


$log P(class = 5|f_1, f_2, ..., f_784)=$ -206.09087174321996

$log P(class = 7|f_1, f_2, ..., f_784)=$ -114.6256618113011

In [10]:
NB = BinNaiveBayes(5)
NB.fit(trainX, trainY)
print('For first test image, k = 5:')
display(Markdown('$log P(class = 5|f_1, f_2, ..., f_784)=$ ' + str(NB.logProb(testX.iloc[0])[5])))
display(Markdown('$log P(class = 7|f_1, f_2, ..., f_784)=$ ' + str(NB.logProb(testX.iloc[0])[7])))

For first test image, k = 5:


$log P(class = 5|f_1, f_2, ..., f_784)=$ -205.91085479090967

$log P(class = 7|f_1, f_2, ..., f_784)=$ -115.0183296837904

### Performance in terms of the classification rate (percentage of all test images correctly classified) for each value of $k$ from $1$ to $5$.

In [11]:
for k in range (1, 6):
    NB = BinNaiveBayes(k)
    NB.fit(trainX, trainY)
    pred = NB.predict(testX)
    print(f'Accuracy score for k={k}: {sklearn.metrics.accuracy_score(testY, pred) * 100}%.')

C:\Users\Dergel\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:61: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return bound(*args, **kwds)


Accuracy score for k=1: 84.27%.
Accuracy score for k=2: 84.25%.
Accuracy score for k=3: 84.19%.
Accuracy score for k=4: 84.17%.
Accuracy score for k=5: 84.11999999999999%.


### Confusion matrix for $k=1$

In [12]:
NB = BinNaiveBayes()
NB.fit(trainX, trainY)
pred = NB.predict(testX)
sklearn.metrics.confusion_matrix(testY, pred)

array([[ 882,    0,    3,    4,    1,   51,   21,    1,   17,    0],
       [   0, 1086,    6,    5,    0,    9,    4,    0,   25,    0],
       [  17,   14,  840,   32,   21,    5,   28,   16,   57,    2],
       [   4,   17,   36,  839,    1,   30,    7,   13,   44,   19],
       [   2,    7,    4,    0,  799,    2,   16,    2,   12,  138],
       [  16,   11,    5,  106,   24,  655,   18,    8,   24,   25],
       [  17,   14,   15,    1,   14,   37,  853,    0,    7,    0],
       [   2,   33,   17,    3,   17,    0,    0,  864,   25,   67],
       [  10,   28,   12,   68,   16,   27,   10,    6,  762,   35],
       [  12,   13,    6,    8,   64,    9,    0,   29,   21,  847]],
      dtype=int64)